In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math

In [3]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'/PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()

['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [7]:
st_interest = Res_eig['stations'][:18]

In [15]:
pdf = Stations.ix[st_interest]

In [16]:
min_long = pdf['longitude'].min()
max_long = pdf['longitude'].max()
min_lat = pdf['latitude'].min()
max_lat = pdf['latitude'].max()
print min_long,max_long,min_lat,max_lat

-88.5172 -85.593 31.5475 32.5844


In [17]:
min_lat,max_lat,min_long,max_long = box = (min_lat,max_lat,min_long,max_long)

In [18]:
pdf.head()

,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
USC00012188,32.3667,-87.6500,70.1,L,DAYTON 1 N,NaN,NaN,NaN
USC00011810,31.7561,-88.1289,14.0,L,COFFEEVILLE L&D,NaN,NaN,NaN
USC00016335,32.1333,-88.0333,6.1,L,PENNINGTON LOCK 2,NaN,NaN,NaN
USC00016334,32.2258,-88.0281,25.9,L,PENNINGTON 2 NE,NaN,NaN,NaN
USW00063874,31.8456,-86.6108,137.5,L,GREENVILLE CRENSHAW AP,NaN,NaN,NaN


In [20]:
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9

m = Map(default_tiles=TileLayer(opacity=1.0), center=center, zoom=zoom)

r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
circles = []
for index,row in pdf.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    # taking sqrt of count so that the  area of the circle corresponds to the count
    c = Circle(location=(_lat,_long), radius=int(300*np.sqrt(100+0.0)), weight=1,
            color='#F00', opacity=0.8, fill_opacity=0.4,
            fill_color='#F00')
    circles.append(c)
    m.add_layer(c)
m   